In [1]:
import osmnx as ox

In [2]:
import numpy as np

In [3]:
from collections import defaultdict

In [4]:
import networkx as nx

In [5]:
from shapely.geometry import Polygon, MultiPolygon

In [14]:
class Name:
    def __init__(self, name: str):
        self.name = name
        
        self.gdf = ox.gdf_from_place(name)
        self.official_name = self.gdf.place_name.values[0]
        self.geometry = self.gdf.geometry[0]
        if not type(self.geometry) is Polygon or type(self.geometry) is MultiPolygon:
            raise TypeError("Location geometry was not a Polygon or a MultiPolygon")
        
    def summary(self):
        print(f"Input Name: {self.name}")
        print(f"Official Name: {self.official_name}")
        print(type(self.geometry))

In [15]:
class Bbox:
    def __init__(self, north, south, east, west):
        self.bbox = (north, south, east, west)
        
    def __iter__(self):
        return (b for b in self.bbox)
    
    def summary(self):
        width = self.bbox[0] - self.bbox[1]
        height = self.bbox[2] - self.bbox[3]
        area = width * height
        print(f"Width: {width}")
        print(f"Height: {height}")
        print(f"Area: {area}")

In [22]:
class NodesGeometry:
    def __init__(self, G, segments):
        self.segments = segments
        nodes_xy = self.create_nodes_xy(G, segments)
        self.nodes = self.create_nodes(nodes_xy)
    
    def create_nodes_xy(self, G, segments):
        nodes_xy = {}
        for node in nodes:
            xy = G.nodes()[node[0]]
            x = xy['x']
            y = xy['y']
            nodes_xy[node] = np.array((x,y))
    
        dist = .00001 #change to 10 feet once you figure out units
        for segment in segments:
            if segment[2]['has_comp']:
                unit_vec = self.segment_unit_vec(segment)
                perp_vec = np.array([unit_vec[1],unit_vec[0]*-1])
                nodes_xy[segment[0]] = nodes_xy[segment[0]] + ((unit_vec * dist) + (perp_vec * dist / 2))
                nodes_xy[segment[1]] = nodes_xy[segment[1]] - ((unit_vec * dist) - (perp_vec * dist / 2))
            else:
                nodes_xy[segment[0]] = nodes_xy[segment[0]] + (unit_vec * dist)
                nodes_xy[segment[1]] = nodes_xy[segment[1]] - (unit_vec * dist)
    
        return nodes_xy
                
    def xy_vec(n1, n2):
        """
        Calculates the vector that takes you from n1 to n2
        """
        xy1 = nodes_xy[n1]
        xy2 = nodes_xy[n2]

        dx = xy2['x'] - xy1['x']
        dy = xy2['y'] - xy1['y']

        return np.array((dx, dy))

    def xy_vec(self, n1, n2):
        """
        Calculates the vector that takes you from n1 to n2
        """
        return nodes_xy[n2] - nodes_xy[n1]

    def segment_vec(self, segment):
        return self.xy_vec(segment[0], segment[1])

    def segment_unit_vec(self, segment):
        arr = self.segment_vec(segment)
        return arr / np.linalg.norm(arr)
    
    def create_nodes(self, nodes_xy):
        nodes_graph = {}
        for k,v in nodes_xy.items():
            nodes_graph[k] = {'x':v[0], 'y':v[1]}
        nodes = [(k, v) for k, v in nodes_graph.items()]
        return nodes

In [23]:
class IntersectionBuilder:
    def __init__(self, in_out):
        """
        :param in_out: in out dictionary of the init_map nodes
        """
        self.intersections = self.create_intersections(in_out)
    
    def create_intersections(self, in_out):
        intersections = []
        for k, v in in_out.items():
            # every in connects to every out - unless same node
            for n_in in v['in']:
                n1 = (k, n_in, 'in')
                for n_out in v['out']:
                    n2 = (k, n_out, 'out')
                    if n_in != n_out:
                        intersections.append((n1, n2, {'type':'intersection'}))
        return intersections

In [24]:
class SegmentBuilder:
    def __init__(self, in_out):
        """
        :param in_out: in out dictionary of the init_map nodes
        """
        self.segments = self.create_segments(in_out)
        self.nodes = self.extract_nodes(self.segments)
    
    def extract_nodes(self, segments):
        nodes = set()
        for segment in segments:
            nodes.add(segment[0])
            nodes.add(segment[1])
        return nodes
        
    def complement_dir(self, s: str):
        """
        TODO: Switch to True and False so I don't have to write this function
        """
        if s == 'in':
            return 'out'
        elif s == 'out':
            return 'in'
        else:
            print("complement_dir failed")
            
    def complement_segment(self, segment):
        """
        Computes the complementary segment of the given segment. The
        complementary segement represents the other direction in a two
        way street.
        """
        n0 = segment[0]
        n1 = segment[1]

        n3 = (n0[0], n0[1], self.complement_dir(n0[2]))
        n2 = (n1[0], n1[1], self.complement_dir(n1[2]))

        return (n2, n3)
    
    def create_segments_set(self, in_out):
        segments_set = set()
        for k, v in in_out.items():
            for node in v['in']:
                n1 = (node, k, 'out')
                n2 = (k, node, 'in')
                segments_set.add((n1, n2))
        return segments_set
    
    def create_segments_list(self, segments):
        segment_list = []
        for segment in segments:
            if (self.complement_segment(segment) in segments):
                has_comp = True
            else:
                has_comp = False
            segment_list.append((segment[0], segment[1], {'type': 'segment', 'has_comp':has_comp}))
        return segment_list
    
    def create_segments(self, in_out):
        segments_set = self.create_segments_set(in_out)
        segments_list = self.create_segments_list(segments_set)
        return segments_list

In [25]:
class GraphBuilder:
    def __init__(self, bound):
        """
        bound: user desired bounds of the graph
        :param bound: Name or Bbox
        """
        self.bound = bound
        self.init_map = self.initialize_map(self.bound)
        self.in_out = self.create_in_out_dict(self.init_map)
        segmentBuilder = SegmentBuilder(self.in_out)
        self.segments = segmentBuilder.segments
        self.nodes = segmentBuilder.nodes
        self.intersections = IntersectionBuilder(self.in_out).intersections
        self.edges = self.create_edges(self.segments, self.intersections)
    
    def initialize_map(self, bound):
        init_map = None
        if type(bound) is Name:
            init_map = ox.graph_from_place(bound.official_name)
        elif type(bound) is Bbox:
            init_map = ox.graph_from_bbox(*bbox)
        else:
            raise RuntimeError("Could not create graph from specified bound")
        return init_map
            
    def plot_map(self, fig_height=10):
        ox.plot_graph(self.init_map, fig_height=fig_height)
                
    def create_in_out_dict(self, G):
        def make_dict():
            return {'in':[],'out':[]}

        in_out = defaultdict(make_dict)
        for start, end in G.edges():
            if start == end:
                continue
            in_out[end]['in'].append(start)
            in_out[start]['out'].append(end)
        return in_out
                        
    def create_edges(self, segments, intersections):
        edges = segments + intersections
        edges = [(u,v,0,d) for u,v,d in edges]
        return edges

In [26]:
bbox = Bbox(38.88300016, 38.878726840000006, -77.09939832, -77.10500768)

In [21]:
GB = GraphBuilder(bbox)

In [ ]:
class MapLoader:
    def __init__(self):
        

In [ ]:
bbox = (38.88300016, 38.878726840000006, -77.09939832, -77.10500768)

In [ ]:
bbox_graph = ox.graph_from_bbox(*bbox)

In [ ]:
ox.plot_graph(bbox_graph, fig_height=3);

In [ ]:
name = Name("Needham, MA")